


### **บทนำ**

**วัตถุประสงค์หลัก:** เพื่อฝึกฝนทักษะสำคัญ 2 ประการที่ได้เรียนรู้จาก Notebooks file = `01_ Structured Data to Insightful.ipynb` และ `03_Structured Outputs_with_Pydatic.ipynb` ได้แก่:

1.  **การสกัดข้อมูล (Structured Data Extraction):** 📝 การแปลงข้อความที่ไม่มีโครงสร้าง (เช่น รีวิว, อีเมล, แผนการเดินทาง) ให้กลายเป็นข้อมูลที่มีโครงสร้างและเป็นระเบียบ โดยใช้ Pydantic Model เป็นตัวกำหนดรูปแบบ
2.  **การสร้างเนื้อหา (Insightful Content Generation):** 🤖 การนำข้อมูลที่มีโครงสร้างที่สกัดมาได้ มาใช้เป็น "วัตถุดิบ" ในการสร้างสรรค์เนื้อหาใหม่ๆ ที่มีความหมายและนำไปใช้ประโยชน์ต่อได้ เช่น การเขียนโพสต์ตอบกลับ, การร่างอีเมล, หรือการจัดทำตารางสรุป

-----

### **โจทย์ข้อที่ 1: ตอบกลับรีวิวร้านกาแฟ ☕**

**สถานการณ์:** คุณเป็นเจ้าของร้านกาแฟที่ใส่ใจลูกค้า และต้องการใช้ AI ช่วยอ่านรีวิวที่มีหลากหลายรูปแบบ จากนั้นนำข้อมูลที่ได้มาสร้างคำขอบคุณที่เป็นส่วนตัวและน่าประทับใจ เพื่อโพสต์ลงโซเชียลมีเดีย

#### **ขั้นตอนการทำ**

**Part 1: สกัดข้อมูลจากรีวิว**

1.  **กำหนดโครงสร้างด้วย Pydantic:** สร้าง Pydantic Model เพื่อบังคับให้ AI สกัดข้อมูลออกมาในรูปแบบที่ต้องการ ดังนี้:
    ```python
    from pydantic import BaseModel, Field
    from typing import List

    class CafeReview(BaseModel):
        cafe_name: str = Field(description="ชื่อของร้านกาแฟที่ถูกรีวิว")
        items_ordered: List[str] = Field(description="ลิสต์ของเมนูที่ลูกค้ารีวิวว่าได้สั่งไป")
        rating: int = Field(description="คะแนนดาวที่ลูกค้าให้ เป็นตัวเลข 1-5")
        comment: str = Field(description="ความคิดเห็นโดยรวมของลูกค้า")
    ```
2.  **เรียกใช้ LLM:** เขียนโค้ดเพื่อส่งข้อความรีวิว (จากชุดข้อมูลทดสอบด้านล่าง) พร้อมกับ `CafeReview` model เข้าไปใน LLM เพื่อให้ AI สกัดข้อมูลออกมาเป็น object ตามโครงสร้างที่กำหนด

**Part 2: สร้างโพสต์ขอบคุณ**

1.  **สร้าง Prompt:** นำ object ที่ได้จาก Part 1 มาสร้างเป็น Prompt ที่มีรายละเอียด เพื่อสั่งให้ AI เขียนแคปชั่น
2.  **เรียกใช้ LLM:** สั่งให้ AI สร้างข้อความตอบกลับที่ดูเป็นธรรมชาติและมีการอ้างอิงถึงเมนูที่ลูกค้าสั่งและคะแนนที่ให้

#### **ข้อมูลสำหรับทดสอบ (Test Cases)**

ใช้ลิสต์ต่อไปนี้เพื่อทดสอบว่าโค้ดของคุณสามารถจัดการกับรีวิวที่หลากหลายได้หรือไม่

```python
queries_cafe_reviews = [
    # Query 1: รีวิวมาตรฐาน ตรงไปตรงมา
    "ไปลองร้าน 'The Reading Room' มาเมื่อวาน สั่งลาเต้ร้อนกับบราวนี่ไปหนึ่งชิ้น กาแฟรสชาติดีมาก แต่บราวนี่เนื้อแน่นไปหน่อย โดยรวมให้ 4 ดาวครับ",

    # Query 2: ใช้ภาษาไม่เป็นทางการ มีการให้คะแนนแบบเศษส่วน
    "เพิ่งกลับมาจากร้าน Bake & Brew คือดีย์มากกก สั่งชาเขียวมัทฉะกับครัวซองต์ไข่เค็มไปคือที่สุด! แต่ร้านเล็กไปนิดนึง คนเยอะเลยต้องรอคิว ให้ 3/5 ดาวละกัน",

    # Query 3: รีวิวเชิงลบ และไม่ได้สั่งเครื่องดื่ม
    "ประสบการณ์ที่ร้าน 'Sip & Smile' ถือว่าน่าผิดหวัง สั่งแค่เค้กมะพร้าวมาชิ้นเดียว แต่รอไป 20 นาที แถมรสชาติก็ธรรมดามาก ไม่ประทับใจเลย ให้ 1 ดาวพอ",

    # Query 4: ไม่ได้บอกชื่อร้านตรงๆ และให้คะแนนแบบบรรยาย
    "วันนี้ไปคาเฟ่เปิดใหม่แถวอารีย์ที่ชื่อ The Hidden Cup มาค่ะ สั่งอเมริกาโน่เย็นกับครัวซองต์แฮมชีส อร่อยทั้งสองอย่างเลยค่ะ ประทับใจมาก เอาไปเลย 5 ดาวเต็ม!",

    # Query 5: ประโยคซับซ้อน มีทั้งคำชมและคำติปนกัน
    "แม้ว่าบรรยากาศที่ร้าน 'Lo-fi Coffee' จะตกแต่งได้สวยงาม และกาแฟ Dirty ที่สั่งไปจะทำออกมาได้ดี แต่ก็ต้องหักคะแนนเรื่องที่จอดรถที่ไม่มีเลย และชีสเค้กหน้าไหม้ที่รสชาติหวานเกินไปมาก สรุปแล้วเลยขอให้แค่ 2 ดาว"
]
```



In [1]:
import IPython
import sys

def clean_notebook():
    IPython.display.clear_output(wait=True)
    print("Notebook cleaned.")

# Install necessary libraries
!pip install openai python-dotenv -q

# Clean up the notebook
clean_notebook()

Notebook cleaned.


In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()


True

In [3]:
queries_cafe_reviews = [
    # Query 1: รีวิวมาตรฐาน ตรงไปตรงมา
    "ไปลองร้าน 'The Reading Room' มาเมื่อวาน สั่งลาเต้ร้อนกับบราวนี่ไปหนึ่งชิ้น กาแฟรสชาติดีมาก แต่บราวนี่เนื้อแน่นไปหน่อย โดยรวมให้ 4 ดาวครับ",

    # Query 2: ใช้ภาษาไม่เป็นทางการ มีการให้คะแนนแบบเศษส่วน
    "เพิ่งกลับมาจากร้าน Bake & Brew คือดีย์มากกก สั่งชาเขียวมัทฉะกับครัวซองต์ไข่เค็มไปคือที่สุด! แต่ร้านเล็กไปนิดนึง คนเยอะเลยต้องรอคิว ให้ 3/5 ดาวละกัน",

    # Query 3: รีวิวเชิงลบ และไม่ได้สั่งเครื่องดื่ม
    "ประสบการณ์ที่ร้าน 'Sip & Smile' ถือว่าน่าผิดหวัง สั่งแค่เค้กมะพร้าวมาชิ้นเดียว แต่รอไป 20 นาที แถมรสชาติก็ธรรมดามาก ไม่ประทับใจเลย ให้ 1 ดาวพอ",

    # Query 4: ไม่ได้บอกชื่อร้านตรงๆ และให้คะแนนแบบบรรยาย
    "วันนี้ไปคาเฟ่เปิดใหม่แถวอารีย์ที่ชื่อ The Hidden Cup มาค่ะ สั่งอเมริกาโน่เย็นกับครัวซองต์แฮมชีส อร่อยทั้งสองอย่างเลยค่ะ ประทับใจมาก เอาไปเลย 5 ดาวเต็ม!",

    # Query 5: ประโยคซับซ้อน มีทั้งคำชมและคำติปนกัน
    "แม้ว่าบรรยากาศที่ร้าน 'Lo-fi Coffee' จะตกแต่งได้สวยงาม และกาแฟ Dirty ที่สั่งไปจะทำออกมาได้ดี แต่ก็ต้องหักคะแนนเรื่องที่จอดรถที่ไม่มีเลย และชีสเค้กหน้าไหม้ที่รสชาติหวานเกินไปมาก สรุปแล้วเลยขอให้แค่ 2 ดาว"
]

queries_cafe_reviews_json = json.dumps(queries_cafe_reviews, indent=2)
queries_cafe_reviews_json

'[\n  "\\u0e44\\u0e1b\\u0e25\\u0e2d\\u0e07\\u0e23\\u0e49\\u0e32\\u0e19 \'The Reading Room\' \\u0e21\\u0e32\\u0e40\\u0e21\\u0e37\\u0e48\\u0e2d\\u0e27\\u0e32\\u0e19 \\u0e2a\\u0e31\\u0e48\\u0e07\\u0e25\\u0e32\\u0e40\\u0e15\\u0e49\\u0e23\\u0e49\\u0e2d\\u0e19\\u0e01\\u0e31\\u0e1a\\u0e1a\\u0e23\\u0e32\\u0e27\\u0e19\\u0e35\\u0e48\\u0e44\\u0e1b\\u0e2b\\u0e19\\u0e36\\u0e48\\u0e07\\u0e0a\\u0e34\\u0e49\\u0e19 \\u0e01\\u0e32\\u0e41\\u0e1f\\u0e23\\u0e2a\\u0e0a\\u0e32\\u0e15\\u0e34\\u0e14\\u0e35\\u0e21\\u0e32\\u0e01 \\u0e41\\u0e15\\u0e48\\u0e1a\\u0e23\\u0e32\\u0e27\\u0e19\\u0e35\\u0e48\\u0e40\\u0e19\\u0e37\\u0e49\\u0e2d\\u0e41\\u0e19\\u0e48\\u0e19\\u0e44\\u0e1b\\u0e2b\\u0e19\\u0e48\\u0e2d\\u0e22 \\u0e42\\u0e14\\u0e22\\u0e23\\u0e27\\u0e21\\u0e43\\u0e2b\\u0e49 4 \\u0e14\\u0e32\\u0e27\\u0e04\\u0e23\\u0e31\\u0e1a",\n  "\\u0e40\\u0e1e\\u0e34\\u0e48\\u0e07\\u0e01\\u0e25\\u0e31\\u0e1a\\u0e21\\u0e32\\u0e08\\u0e32\\u0e01\\u0e23\\u0e49\\u0e32\\u0e19 Bake & Brew \\u0e04\\u0e37\\u0e2d\\u0e14\\u0e35\\u0e22\\u0e4

In [4]:
from pydantic import BaseModel, Field
from typing import List

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"    


class CafeReview(BaseModel):
        cafe_name: str = Field(description="ชื่อของร้านกาแฟที่ถูกรีวิว")
        items_ordered: List[str] = Field(description="ลิสต์ของเมนูที่ลูกค้ารีวิวว่าได้สั่งไป")
        rating: int = Field(description="คะแนนดาวที่ลูกค้าให้ เป็นตัวเลข 1-5")
        comment: str = Field(description="ความคิดเห็นโดยรวมของลูกค้า")
        


In [6]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List
review_list  = []


for index, review in enumerate(queries_cafe_reviews):
    response = client.beta.chat.completions.parse(
        model= model_name,
        messages=[
            {"role": "system", "content": "Extract the cafe review information"},
            {"role": "user","content": f"{review}"},
        ],
        temperature=0.7,
        response_format=CafeReview,
        )
    event = response.choices[0].message.parsed
    review_data = event.model_dump()  # ผลลัพธ์จากการแปลงโมเดล
    review_list.append(review_data) # เพิ่มข้อมูลลงใน list
    
    for field, value in event.model_dump().items():
        print(f"{field}: {value}")


cafe_name: Lo-fi Coffee
items_ordered: ['กาแฟ Dirty', 'ชีสเค้กหน้าไหม้']
rating: 2
comment: บรรยากาศร้านสวยงาม กาแฟ Dirty อร่อย แต่ไม่มีที่จอดรถ และชีสเค้กหน้าไหม้หวานเกินไป


In [10]:
response = client.beta.chat.completions.parse(
        model= model_name,
        messages=[
            {"role": "system", "content": """
ณเป็นเจ้าของร้านกาแฟที่มีความใส่ใจลูกค้า และมีความสามารถในการตอบกลับรีวิวจากลูกค้าอย่างเป็นมิตรและมีประสิทธิภาพโดยคุณจะต้องตอบกลับรีวิวที่ลูกค้าเขียนไว้ด้วยการขอบคุณลูกค้าสำหรับความคิดเห็น พร้อมกับอ้างอิงถึงเมนูที่ลูกค้าสั่งและคะแนนที่ให้ในรีวิวคำตอบที่สร้างต้องดูเป็นธรรมชาติ, เป็นกันเองและต้องคำนึงถึงความคิดเห็นเชิงบวกและเชิงลบที่ลูกค้าระบุ
"""},
            {"role": "user","content": f"""สร้างข้อความตอบกลับที่ดูเป็นธรรมชาติและมีการอ้างอิงถึงเมนูที่ลูกค้าสั่งและคะแนนที่ให้ 
             จากข้อมูลลูกค้าต่่อไปนี้
             
             {review_list}
             """
             
             
             },
        ],
        temperature=0.7,
        )
event = response.choices[0].message.content
print(event)

แน่นอนค่ะ! ต่อไปนี้คือข้อความตอบกลับรีวิวแต่ละร้านที่เป็นธรรมชาติ อ้างอิงถึงเมนูที่สั่งและคะแนนที่ลูกค้าให้ พร้อมใส่ใจทั้งความคิดเห็นเชิงบวกและเชิงลบ

---

### 1. The Reading Room

ขอบคุณมากนะคะที่แวะมาที่ The Reading Room และสำหรับรีวิว 4 ดาวด้วยค่ะ ดีใจที่คุณชอบลาเต้ร้อนของเรา ส่วนเรื่องบราวนี่ที่เนื้อแน่นไปหน่อย ทางร้านจะนำไปปรับปรุงให้ดียิ่งขึ้นค่ะ หวังว่าจะได้ต้อนรับคุณอีกครั้งนะคะ และถ้าคราวหน้ามีเมนูไหนอยากแนะนำเพิ่มเติม แจ้งเราได้เลยค่ะ

---

### 2. Bake & Brew

ขอบคุณสำหรับรีวิวและคะแนน 3 ดาวนะคะ ดีใจที่ชาเขียวมัทฉะกับครัวซองต์ไข่เค็มถูกใจคุณมากๆ ต้องขออภัยที่ร้านเล็กและลูกค้าเยอะจนต้องรอคิวนานค่ะ ทางร้านจะพยายามจัดการคิวและบริการให้ดียิ่งขึ้น หวังว่าจะได้ต้อนรับคุณอีกในโอกาสหน้าค่ะ

---

### 3. Sip & Smile

ขอบคุณมากนะคะที่แวะมาทานเค้กมะพร้าวที่ร้าน และสำหรับความคิดเห็นตรงไปตรงมาค่ะ ต้องขออภัยเป็นอย่างสูงที่ทำให้ต้องรอนานถึง 20 นาที และรสชาติยังไม่ประทับใจ ทางร้านจะนำข้อเสนอแนะของคุณไปปรับปรุงทั้งเรื่องบริการและคุณภาพของเค้กค่ะ หวังว่าจะมีโอกาสได้ดูแลคุณอีกครั้งและมอบประสบการ



### **โจทย์ข้อที่ 2: สรุปรายการสั่งซื้อของลูกค้า 🛍️**

**สถานการณ์:** ลูกค้าส่งอีเมลสั่งซื้อของหลายรายการในรูปแบบที่แตกต่างกัน คุณต้องใช้ AI ช่วยแปลงอีเมลเหล่านั้นให้เป็นข้อมูลที่มีโครงสร้าง เพื่อให้ง่ายต่อการเตรียมจัดของและส่งอีเมลยืนยันกลับไปหาลูกค้า

#### **ขั้นตอนการทำ**

**Part 1: สกัดข้อมูลรายการสั่งซื้อ**

1.  **กำหนดโครงสร้างแบบซ้อนกัน:** สร้าง Pydantic Model 2 ชั้น เพื่อจัดการกับข้อมูลที่มีความสัมพันธ์แบบหนึ่งต่อกลุ่ม (One-to-Many)
    ```python
    from pydantic import BaseModel, Field
    from typing import List

    class OrderItem(BaseModel):
        item_name: str = Field(description="ชื่อสินค้าที่สั่ง")
        quantity: int = Field(description="จำนวนสินค้าชิ้นนั้นๆ")

    class Order(BaseModel):
        customer_name: str = Field(description="ชื่อของลูกค้าผู้สั่งซื้อ")
        items: List[OrderItem] = Field(description="ลิสต์ของสินค้าทั้งหมดในออเดอร์")
    ```
2.  **เรียกใช้ LLM:** ส่งเนื้อหาอีเมล (จากชุดข้อมูลทดสอบ) พร้อมกับ `Order` model เข้าไปเพื่อให้ AI แปลงข้อมูลเป็น object ที่ถูกต้อง

**Part 2: สร้างอีเมลยืนยัน**

1.  **เตรียมข้อมูลสำหรับ Prompt:** เขียนโค้ดเพื่อวนลูป `items` ที่สกัดมาได้ เพื่อสร้างเป็นรายการสินค้า (string) และคำนวณจำนวนสินค้ารวม
2.  **สร้าง Prompt และเรียกใช้ LLM:** สั่งให้ AI ร่างอีเมลยืนยันการสั่งซื้อ โดยใช้ชื่อลูกค้า, รายการสินค้า, และจำนวนรวมที่เตรียมไว้

#### **ข้อมูลสำหรับทดสอบ (Test Cases)**

```python
queries_customer_orders = [
    # Query 1: รูปแบบมาตรฐาน ใช้ขีดนำหน้า
    "สวัสดีครับ ผมมานะ ต้องการสั่งของตามนี้ครับ:\n- หูฟังไร้สาย XYZ จำนวน 1 อัน\n- เคสกันกระแทก สีดำ 2 ชิ้น\n- สายชาร์จ USB-C ยาว 2 เมตร 1 เส้น",

    # Query 2: บอกจำนวนในประโยค และชื่ออยู่ท้ายอีเมล
    "รบกวนสั่งของหน่อยนะคะ อยากได้เมาส์ไร้สายสีชมพูหนึ่งอัน แล้วก็แผ่นรองเมาส์ลายแมวอีกสองแผ่นค่ะ ขอบคุณค่ะ - จากคุณวันดี",

    # Query 3: มีสินค้าที่ไม่ระบุจำนวน (ให้ตีความเป็น 1) และมีคำถามปนมา
    "สนใจสั่งคีย์บอร์ด Mechanical รุ่น K-1200 และขอพวงกุญแจลายอวกาศด้วยค่ะ ไม่แน่ใจว่าต้องสั่งขั้นต่ำเท่าไหร่คะ? ถ้าไม่ต้อง รบกวนจัดส่งได้เลย ชื่อผู้รับคือสมใจค่ะ",

    # Query 4: ระบุจำนวนด้วยตัวเลขในวงเล็บและคั่นด้วยจุลภาค
    "ผมปีเตอร์นะครับ ขอสั่ง, ครีมกันแดด (2), โฟมล้างหน้า (1), และสเปรย์น้ำแร่ (3) ครับ",

    # Query 5: รายการสั่งซื้อที่ซับซ้อนและมีการแก้ไข
    "ถึงแอดมิน ผมวิรัชครับ ขอสั่งเสื้อโปโลสีขาว 2 ตัวครับ ขนาด L นะครับ แล้วก็ขอกางเกงขาสั้นสีเทา 1 ตัว ขนาด M อ้อ! เกือบลืม เพิ่มถุงเท้าสีขาว 3 คู่ด้วยครับ"
]
```


In [11]:
queries_customer_orders = [
    # Query 1: รูปแบบมาตรฐาน ใช้ขีดนำหน้า
    "สวัสดีครับ ผมมานะ ต้องการสั่งของตามนี้ครับ:\n- หูฟังไร้สาย XYZ จำนวน 1 อัน\n- เคสกันกระแทก สีดำ 2 ชิ้น\n- สายชาร์จ USB-C ยาว 2 เมตร 1 เส้น",

    # Query 2: บอกจำนวนในประโยค และชื่ออยู่ท้ายอีเมล
    "รบกวนสั่งของหน่อยนะคะ อยากได้เมาส์ไร้สายสีชมพูหนึ่งอัน แล้วก็แผ่นรองเมาส์ลายแมวอีกสองแผ่นค่ะ ขอบคุณค่ะ - จากคุณวันดี",

    # Query 3: มีสินค้าที่ไม่ระบุจำนวน (ให้ตีความเป็น 1) และมีคำถามปนมา
    "สนใจสั่งคีย์บอร์ด Mechanical รุ่น K-1200 และขอพวงกุญแจลายอวกาศด้วยค่ะ ไม่แน่ใจว่าต้องสั่งขั้นต่ำเท่าไหร่คะ? ถ้าไม่ต้อง รบกวนจัดส่งได้เลย ชื่อผู้รับคือสมใจค่ะ",

    # Query 4: ระบุจำนวนด้วยตัวเลขในวงเล็บและคั่นด้วยจุลภาค
    "ผมปีเตอร์นะครับ ขอสั่ง, ครีมกันแดด (2), โฟมล้างหน้า (1), และสเปรย์น้ำแร่ (3) ครับ",

    # Query 5: รายการสั่งซื้อที่ซับซ้อนและมีการแก้ไข
    "ถึงแอดมิน ผมวิรัชครับ ขอสั่งเสื้อโปโลสีขาว 2 ตัวครับ ขนาด L นะครับ แล้วก็ขอกางเกงขาสั้นสีเทา 1 ตัว ขนาด M อ้อ! เกือบลืม เพิ่มถุงเท้าสีขาว 3 คู่ด้วยครับ"
]

# queries_customer_orders_json = json.dumps(queries_customer_orders, indent=2)
# queries_customer_orders_json

'[\n  "\\u0e2a\\u0e27\\u0e31\\u0e2a\\u0e14\\u0e35\\u0e04\\u0e23\\u0e31\\u0e1a \\u0e1c\\u0e21\\u0e21\\u0e32\\u0e19\\u0e30 \\u0e15\\u0e49\\u0e2d\\u0e07\\u0e01\\u0e32\\u0e23\\u0e2a\\u0e31\\u0e48\\u0e07\\u0e02\\u0e2d\\u0e07\\u0e15\\u0e32\\u0e21\\u0e19\\u0e35\\u0e49\\u0e04\\u0e23\\u0e31\\u0e1a:\\n- \\u0e2b\\u0e39\\u0e1f\\u0e31\\u0e07\\u0e44\\u0e23\\u0e49\\u0e2a\\u0e32\\u0e22 XYZ \\u0e08\\u0e33\\u0e19\\u0e27\\u0e19 1 \\u0e2d\\u0e31\\u0e19\\n- \\u0e40\\u0e04\\u0e2a\\u0e01\\u0e31\\u0e19\\u0e01\\u0e23\\u0e30\\u0e41\\u0e17\\u0e01 \\u0e2a\\u0e35\\u0e14\\u0e33 2 \\u0e0a\\u0e34\\u0e49\\u0e19\\n- \\u0e2a\\u0e32\\u0e22\\u0e0a\\u0e32\\u0e23\\u0e4c\\u0e08 USB-C \\u0e22\\u0e32\\u0e27 2 \\u0e40\\u0e21\\u0e15\\u0e23 1 \\u0e40\\u0e2a\\u0e49\\u0e19",\n  "\\u0e23\\u0e1a\\u0e01\\u0e27\\u0e19\\u0e2a\\u0e31\\u0e48\\u0e07\\u0e02\\u0e2d\\u0e07\\u0e2b\\u0e19\\u0e48\\u0e2d\\u0e22\\u0e19\\u0e30\\u0e04\\u0e30 \\u0e2d\\u0e22\\u0e32\\u0e01\\u0e44\\u0e14\\u0e49\\u0e40\\u0e21\\u0e32\\u0e2a\\u0e4c\\u0e44\\u0e23\\u0e49\\u0

In [12]:
from pydantic import BaseModel, Field
from typing import List

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"    


class OrderItem(BaseModel):
        item_name: str = Field(description="ชื่อสินค้าที่สั่ง")
        quantity: int = Field(description="จำนวนสินค้าชิ้นนั้นๆ")

class Order(BaseModel):
        customer_name: str = Field(description="ชื่อของลูกค้าผู้สั่งซื้อ")
        items: List[OrderItem] = Field(description="ลิสต์ของสินค้าทั้งหมดในออเดอร์")
        


In [17]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List
customer_orders_list  = []


for index, order in enumerate(queries_customer_orders):
    response = client.beta.chat.completions.parse(
        model= model_name,
        messages=[
            {"role": "system", "content": "จัดการกับข้อมูลที่มีความสัมพันธ์แบบหนึ่งต่อกลุ่ม (One-to-Many)"},
            {"role": "user","content": f"{order}"},
        ],
        temperature=0.7,
        response_format=Order,
        )
    event = response.choices[0].message.parsed
    customer_orders = event.model_dump()  # ผลลัพธ์จากการแปลงโมเดล
    customer_orders_list.append(customer_orders) # เพิ่มข้อมูลลงใน list
    
    for field, value in event.model_dump().items():
        print(f"{field}: {value}")

customer_name: มานะ
items: [{'item_name': 'หูฟังไร้สาย XYZ', 'quantity': 1}, {'item_name': 'เคสกันกระแทก สีดำ', 'quantity': 2}, {'item_name': 'สายชาร์จ USB-C ยาว 2 เมตร', 'quantity': 1}]
customer_name: คุณวันดี
items: [{'item_name': 'เมาส์ไร้สายสีชมพู', 'quantity': 1}, {'item_name': 'แผ่นรองเมาส์ลายแมว', 'quantity': 2}]
customer_name: สมใจ
items: [{'item_name': 'คีย์บอร์ด Mechanical รุ่น K-1200', 'quantity': 1}, {'item_name': 'พวงกุญแจลายอวกาศ', 'quantity': 1}]
customer_name: ปีเตอร์
items: [{'item_name': 'ครีมกันแดด', 'quantity': 2}, {'item_name': 'โฟมล้างหน้า', 'quantity': 1}, {'item_name': 'สเปรย์น้ำแร่', 'quantity': 3}]
customer_name: วิรัช
items: [{'item_name': 'เสื้อโปโลสีขาว ขนาด L', 'quantity': 2}, {'item_name': 'กางเกงขาสั้นสีเทา ขนาด M', 'quantity': 1}, {'item_name': 'ถุงเท้าสีขาว', 'quantity': 3}]


In [24]:
# ฟังก์ชันสำหรับเตรียมข้อมูลรายการสินค้าและคำนวณจำนวนสินค้ารวม
for order in customer_orders_list:
    customer_name = order['customer_name']  # ชื่อลูกค้า
    items = order['items']  # รายการสินค้าที่ลูกค้าสั่ง
    
    # สร้างรายการสินค้า (string)
    item_list = [f"{item['item_name']} (จำนวน: {item['quantity']})" for item in items]
    
    # คำนวณจำนวนสินค้ารวม
    total_items = sum(item['quantity'] for item in items)
    
    
    response = client.beta.chat.completions.parse(
        model= model_name,
        messages=[
            {"role": "system", "content": """คุณเป็น AI ที่ช่วยเขียนอีเมลยืนยันการสั่งซื้อจากลูกค้า"""},
            {"role": "user","content": f"""ร่างอีเมลยืนยันการสั่งซื้อ โดยใช้ชื่อลูกค้า, รายการสินค้า, และจำนวนรวมที่เตรียมไว้
             ข้อมูลมีดังนี้
             {item_list} {total_items} {customer_name}
             """},
        ],
        temperature=0.7,
        )
    email_content  = response.choices[0].message.content
    print(email_content)
    print("="*80)
    # พิมพ์ข้อมูลรายการสินค้าและจำนวนรวม
    # print(f"ชื่อลูกค้า: {customer_name}")
    # print("รายการสินค้า:")
    # for item in item_list:
    #     print(item)
    # print(f"จำนวนสินค้ารวม: {total_items}\n")
    


เรื่อง: ยืนยันการสั่งซื้อสินค้า

เรียน คุณมานะ

บริษัทขอขอบพระคุณที่ท่านได้สั่งซื้อสินค้ากับทางเรา ทางบริษัทขอยืนยันการสั่งซื้อของท่าน ดังรายละเอียดต่อไปนี้

รายการสินค้า:  
1. หูฟังไร้สาย XYZ (จำนวน: 1)  
2. เคสกันกระแทก สีดำ (จำนวน: 2)  
3. สายชาร์จ USB-C ยาว 2 เมตร (จำนวน: 1)

จำนวนสินค้าทั้งหมด: 4 ชิ้น

หากข้อมูลการสั่งซื้อถูกต้อง ทางเราจะดำเนินการจัดส่งสินค้าให้ท่านโดยเร็วที่สุด  
หากต้องการแก้ไขหรือสอบถามข้อมูลเพิ่มเติม กรุณาติดต่อกลับที่อีเมลนี้หรือโทร [เบอร์ติดต่อ]

ขอขอบพระคุณที่ไว้วางใจในบริการของเรา

ขอแสดงความนับถือ  
[ชื่อบริษัท/ทีมงาน]
หัวเรื่อง: ยืนยันการสั่งซื้อสินค้า

เรียน คุณวันดี

ขอขอบคุณที่สั่งซื้อสินค้ากับทางร้านของเรา ทางเราขอแจ้งยืนยันการสั่งซื้อของคุณ โดยมีรายละเอียดดังนี้

รายการสินค้า
1. เมาส์ไร้สายสีชมพู (จำนวน: 1)
2. แผ่นรองเมาส์ลายแมว (จำนวน: 2)

รวมจำนวนสินค้าทั้งหมด: 3 ชิ้น

ทางเราจะดำเนินการจัดเตรียมและจัดส่งสินค้าให้คุณวันดีโดยเร็ว หากมีข้อสงสัยหรือต้องการแก้ไขข้อมูลใด ๆ กรุณาติดต่อกลับทางอีเมลหรือโทรศัพท์ตามที่แจ้งไว้

ขอขอบพระคุณที่ไว้วางใจใช้บริการ




### **โจทย์ข้อที่ 3: จัดตารางเที่ยว 🗺️**

**สถานการณ์:** เพื่อนส่งแผนการเดินทางมาให้ในรูปแบบข้อความบรรยาย ซึ่งอาจจะอ่านยากและสับสน คุณต้องใช้ AI ช่วยสกัดข้อมูลเวลาและกิจกรรมตามลำดับ แล้วจัดทำเป็นตารางแผนเที่ยวที่ชัดเจนและอ่านง่าย

#### **ขั้นตอนการทำ**

**Part 1: สกัดข้อมูลแผนการเดินทาง**

1.  **กำหนดโครงสร้างสำหรับข้อมูลที่มีลำดับ:** สร้าง Pydantic Model เพื่อเก็บข้อมูลของแต่ละกิจกรรมตามลำดับเวลา
    ```python
    from pydantic import BaseModel, Field
    from typing import List

    class ScheduleItem(BaseModel):
        time: str = Field(description="เวลาของกิจกรรม")
        activity: str = Field(description="รายละเอียดของกิจกรรมที่จะทำ")

    class Itinerary(BaseModel):
        plan: List[ScheduleItem] = Field(description="ลิสต์ของแผนการเดินทางที่เรียงตามลำดับเวลา")
    ```
2.  **เรียกใช้ LLM:** ส่งข้อความแผนเที่ยว (จากชุดข้อมูลทดสอบ) พร้อม `Itinerary` model เข้าไป เพื่อให้ AI สกัดและเรียงลำดับกิจกรรมให้ถูกต้อง

**Part 2: สร้างตารางแผนเที่ยว**

1.  **สร้าง Prompt สำหรับการจัดรูปแบบ:** สั่งให้ LLM นำข้อมูล `plan` ที่เป็นลิสต์ของ object มาแปลงให้อยู่ในรูปแบบตาราง Markdown
2.  **เรียกใช้ LLM:** รับผลลัพธ์เป็น String ที่เป็นโค้ด Markdown สำหรับสร้างตาราง

#### **ข้อมูลสำหรับทดสอบ (Test Cases)**

```python
queries_travel_plans = [
    # Query 1: รูปแบบมาตรฐาน มีเวลาชัดเจน
    "ทริปหัวหินวันเดียว: 10:00 น. ไปถึงสถานีรถไฟหัวหิน ถ่ายรูปเล่น, 12:30 น. กินข้าวเที่ยงที่ร้านเจ๊เขียวซีฟู้ด, 14:00 น. เดินทางไปตลาดซิเคด้า, 18:00 น. เดินทางกลับ",

    # Query 2: ใช้การบอกลำดับและช่วงเวลา (เช้า, บ่าย, เย็น)
    "แผนเที่ยวเชียงรายของเรานะ เริ่มตอนเช้าไปวัดร่องขุ่นก่อนเลย หลังจากนั้นค่อยไปหาข้าวกินตอนกลางวัน บ่ายๆ ว่าจะแวะไปไร่ชาฉุยฟง แล้วปิดท้ายตอนเย็นที่สิงห์ปาร์ค",

    # Query 3: เล่าแบบไม่เรียงตามลำดับเวลา
    "โอเค สรุปแพลนน่านนะ ตอนเย็นเราจะไปเดินถนนคนเดินกัน แต่ว่าตอนบ่ายโมงเราต้องไปไหว้พระธาตุแช่แห้งก่อนนะ ส่วนตอนเช้าสุดเลย พอไปถึงเราจะไปวัดภูมินทร์กันก่อนเลย ประมาณ 9 โมง",

    # Query 4: มีกิจกรรมย่อยซ้อนอยู่ในกิจกรรมหลัก
    "แพลนสำหรับวันพรุ่งนี้ เริ่ม 9 โมงเช้า เจอกันที่ท่าเรือเพื่อไปเกาะล้าน พอไปถึงแล้ว กิจกรรมแรกคือดำน้ำดูปะการัง จากนั้นประมาณบ่ายโมงค่อยกินข้าวเที่ยงกันที่หาดตาแหวน แล้วค่อยเดินทางกลับตอนสี่โมงเย็น",

    # Query 5: ใช้ภาษาพูดและไม่มีเวลาที่ชัดเจนมากนัก
    "สรุปนะเพื่อนๆ เราไปถึงเขาใหญ่กันก่อนเที่ยง หาข้าวเที่ยงกินกันแถวนั้นแหละ พอกินเสร็จก็เข้าที่พักเก็บของ บ่ายแก่ๆ ค่อยออกไปเที่ยวคาเฟ่สวยๆ ซักที่นึง ตกเย็นค่อยไปกินหมูกระทะกัน"
]
```

In [27]:
queries_travel_plans = [
    # Query 1: รูปแบบมาตรฐาน มีเวลาชัดเจน
    "ทริปหัวหินวันเดียว: 10:00 น. ไปถึงสถานีรถไฟหัวหิน ถ่ายรูปเล่น, 12:30 น. กินข้าวเที่ยงที่ร้านเจ๊เขียวซีฟู้ด, 14:00 น. เดินทางไปตลาดซิเคด้า, 18:00 น. เดินทางกลับ",

    # Query 2: ใช้การบอกลำดับและช่วงเวลา (เช้า, บ่าย, เย็น)
    "แผนเที่ยวเชียงรายของเรานะ เริ่มตอนเช้าไปวัดร่องขุ่นก่อนเลย หลังจากนั้นค่อยไปหาข้าวกินตอนกลางวัน บ่ายๆ ว่าจะแวะไปไร่ชาฉุยฟง แล้วปิดท้ายตอนเย็นที่สิงห์ปาร์ค",

    # Query 3: เล่าแบบไม่เรียงตามลำดับเวลา
    "โอเค สรุปแพลนน่านนะ ตอนเย็นเราจะไปเดินถนนคนเดินกัน แต่ว่าตอนบ่ายโมงเราต้องไปไหว้พระธาตุแช่แห้งก่อนนะ ส่วนตอนเช้าสุดเลย พอไปถึงเราจะไปวัดภูมินทร์กันก่อนเลย ประมาณ 9 โมง",

    # Query 4: มีกิจกรรมย่อยซ้อนอยู่ในกิจกรรมหลัก
    "แพลนสำหรับวันพรุ่งนี้ เริ่ม 9 โมงเช้า เจอกันที่ท่าเรือเพื่อไปเกาะล้าน พอไปถึงแล้ว กิจกรรมแรกคือดำน้ำดูปะการัง จากนั้นประมาณบ่ายโมงค่อยกินข้าวเที่ยงกันที่หาดตาแหวน แล้วค่อยเดินทางกลับตอนสี่โมงเย็น",

    # Query 5: ใช้ภาษาพูดและไม่มีเวลาที่ชัดเจนมากนัก
    "สรุปนะเพื่อนๆ เราไปถึงเขาใหญ่กันก่อนเที่ยง หาข้าวเที่ยงกินกันแถวนั้นแหละ พอกินเสร็จก็เข้าที่พักเก็บของ บ่ายแก่ๆ ค่อยออกไปเที่ยวคาเฟ่สวยๆ ซักที่นึง ตกเย็นค่อยไปกินหมูกระทะกัน"
]

In [28]:
from pydantic import BaseModel, Field
from typing import List

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4.1"    

class ScheduleItem(BaseModel):
    time: str = Field(description="เวลาของกิจกรรม")
    activity: str = Field(description="รายละเอียดของกิจกรรมที่จะทำ")

class Itinerary(BaseModel):
    plan: List[ScheduleItem] = Field(description="ลิสต์ของแผนการเดินทางที่เรียงตามลำดับเวลา")

In [29]:
from openai import OpenAI
from pydantic import BaseModel
from typing import List
travel_list  = []


for index, plans in enumerate(queries_travel_plans):
    response = client.beta.chat.completions.parse(
        model= model_name,
        messages=[
            {"role": "system", "content": "สกัดและเรียงลำดับกิจกรรมให้ถูกต้อง"},
            {"role": "user","content": f"{plans}"},
        ],
        temperature=0.7,
        response_format=Itinerary,
        )
    event = response.choices[0].message.parsed
    travel_plans = event.model_dump()  # ผลลัพธ์จากการแปลงโมเดล
    travel_list.append(travel_plans) # เพิ่มข้อมูลลงใน list
    
    for field, value in event.model_dump().items():
        print(f"{field}: {value}")

plan: [{'time': '10:00 น.', 'activity': 'ไปถึงสถานีรถไฟหัวหิน ถ่ายรูปเล่น'}, {'time': '12:30 น.', 'activity': 'กินข้าวเที่ยงที่ร้านเจ๊เขียวซีฟู้ด'}, {'time': '14:00 น.', 'activity': 'เดินทางไปตลาดซิเคด้า'}, {'time': '18:00 น.', 'activity': 'เดินทางกลับ'}]
plan: [{'time': 'เช้า', 'activity': 'ไปวัดร่องขุ่น'}, {'time': 'กลางวัน', 'activity': 'หาข้าวกินมื้อกลางวัน'}, {'time': 'บ่าย', 'activity': 'แวะไร่ชาฉุยฟง'}, {'time': 'เย็น', 'activity': 'เที่ยวสิงห์ปาร์ค'}]
plan: [{'time': '09:00', 'activity': 'ไปวัดภูมินทร์'}, {'time': '13:00', 'activity': 'ไหว้พระธาตุแช่แห้ง'}, {'time': 'เย็น', 'activity': 'เดินถนนคนเดิน'}]
plan: [{'time': '09:00', 'activity': 'เจอกันที่ท่าเรือเพื่อเดินทางไปเกาะล้าน'}, {'time': '10:00', 'activity': 'เดินทางถึงเกาะล้าน เตรียมตัวสำหรับกิจกรรม'}, {'time': '10:30', 'activity': 'ดำน้ำดูปะการัง'}, {'time': '13:00', 'activity': 'กินข้าวเที่ยงที่หาดตาแหวน'}, {'time': '16:00', 'activity': 'เดินทางกลับจากเกาะล้าน'}]
plan: [{'time': 'ก่อนเที่ยง', 'activity': 'เดินทางถึงเขาใหญ

In [31]:
# ฟังก์ชันสำหรับเตรียมข้อมูลรายการสินค้าและคำนวณจำนวนสินค้ารวม
    
    
response = client.beta.chat.completions.parse(
    model= model_name,
    messages=[
        {"role": "system", "content": """คุณคือผู้ช่วยในการจัดการแผนการเดินทาง ให้ข้อมูลแผนเที่ยวที่มีลำดับเวลาและกิจกรรมต่างๆ มาในรูปแบบที่อ่านง่าย โดยแปลงข้อมูลเป็นตาราง Markdown ที่มีคอลัมน์ 'เวลา' และ 'กิจกรรม' เพื่อให้สามารถอ่านและเข้าใจได้ง่าย
        โปรดตรวจสอบให้แน่ใจว่าแผนการเดินทางถูกจัดเรียงตามลำดับเวลาอย่างถูกต้อง"""},
        {"role": "user","content": f"""โปรดจัดทำตารางแผนการเดินทางสำหรับวันที่ ตามข้อมูลด้านล่างนี้ให้อยู่ในรูปแบบ Markdown
        ข้อมูล {travel_list}
        """},
    ],
    temperature=0.7,
    )
markdown = response.choices[0].message.content
print(markdown)
print("="*80)
    # พิมพ์ข้อมูลรายการสินค้าและจำนวนรวม
    # print(f"ชื่อลูกค้า: {customer_name}")
    # print("รายการสินค้า:")
    # for item in item_list:
    #     print(item)
    # print(f"จำนวนสินค้ารวม: {total_items}\n")

### วันที่ 1

| เวลา       | กิจกรรม                                 |
|------------|------------------------------------------|
| 10:00 น.   | ไปถึงสถานีรถไฟหัวหิน ถ่ายรูปเล่น           |
| 12:30 น.   | กินข้าวเที่ยงที่ร้านเจ๊เขียวซีฟู้ด           |
| 14:00 น.   | เดินทางไปตลาดซิเคด้า                     |
| 18:00 น.   | เดินทางกลับ                              |

---

### วันที่ 2

| เวลา     | กิจกรรม                     |
|----------|------------------------------|
| เช้า     | ไปวัดร่องขุ่น                 |
| กลางวัน  | หาข้าวกินมื้อกลางวัน           |
| บ่าย     | แวะไร่ชาฉุยฟง                |
| เย็น     | เที่ยวสิงห์ปาร์ค              |

---

### วันที่ 3

| เวลา     | กิจกรรม                   |
|----------|----------------------------|
| 09:00    | ไปวัดภูมินทร์                |
| 13:00    | ไหว้พระธาตุแช่แห้ง           |
| เย็น     | เดินถนนคนเดิน                |

---

### วันที่ 4

| เวลา     | กิจกรรม                                |
|----------|-------------------------